# solrf fieldmaps

Create solrf fieldmaps from on-axis data

In [ ]:
import numpy as np
from numpy import pi

import matplotlib.pyplot as plt

# Original data

In [ ]:
# Here is some on-axis data
BDAT = "templates/solenoid/newSOL.dat"

In [ ]:
DAT = np.loadtxt(BDAT).T
Z, BZ = DAT[0], DAT[1] / DAT[1].max()

# Test odd number of points
Z = Z[:-1]
BZ = BZ[:-1]

SKIP = 1  # Thin out data for visualization

Z = Z[::SKIP]
BZ = BZ[::SKIP]

# Get spacing
DZ = np.diff(Z)
assert np.allclose(DZ, DZ[0])
DZ = DZ[0]
L = np.ptp(Z)


# BZ *= sin(100*Z)

# Test for odd number of points

BZ[-1] = BZ[0]  # assert periodicity

plt.plot(Z, BZ, marker=".")
plt.title(f"{len(Z)} points")

# RFcoef Fortran program

In [ ]:
from pmd_beamphysics.interfaces.impact import create_fourier_coefficients

N_COEF = 20

fcoefs1 = create_fourier_coefficients(Z, BZ, n=N_COEF)

In [ ]:
from impact.fieldmaps import run_RFcoef

res = run_RFcoef(Z, BZ, n_coef=N_COEF, exe="$GITHUB/IMPACT-T/utilities/RFcoeflcls")

rfdatax = res["rfdatax"]
rfdatax2 = res["rfdatax2"]
rfdataout = res["rfdata.out"]
rfdatax, len(rfdatax2)

# Reconstruction


The coefficients in these files are defined so that a field $E$ can be reconstructed at position $z$ as 

$ E(z) = A_0 + \sum_{n=1}^{N} A_n \cos\left(2\pi n \frac{z}{L} \right) + B_n \sin\left(2\pi n \frac{z}{L} \right)$

where $z~\epsilon~[-L/2, L/2]$

This function will handle this efficiently.

In [ ]:
from pmd_beamphysics.interfaces.impact import fourier_field_reconsruction

In [ ]:
@np.vectorize
def f0(z):
    return fourier_field_reconsruction(z, rfdatax, z0=Z.min(), zlen=L)


rBZ0 = f0(Z)
errx = rfdataout[:, 1] - BZ
err0 = rBZ0 - BZ

fig, ax = plt.subplots()
# ax.plot(rBZ0, label='reconstructed')
ax.plot(errx, label="rfdata.out")
ax.plot(err0, "--", label="Python on rfdatax")
ax.legend()
ax.set_ylabel("reconstruction error")

# Create coefficients via FFT

In [ ]:
from pmd_beamphysics.interfaces.impact import create_fourier_coefficients_via_fft

In [ ]:
fcoefs1 = create_fourier_coefficients_via_fft(BZ, n_coef=N_COEF)


@np.vectorize
def f1(z):
    return fourier_field_reconsruction(z, fcoefs1, z0=Z.min(), zlen=L)


rBZ1 = f1(Z)
err1 = rBZ1 - BZ

plt.plot(errx, label="rfdata.out")
plt.plot(err0, "--", label="Python on rfdatax")
plt.plot(err1, "--", label="Python on FFT-created coefs")
plt.legend()
plt.ylabel("reconstruction error")

In [ ]:
from impact import fieldmaps

?fieldmaps.create_fourier_coefficients

# Create LUME-Impact style fieldmap

In [ ]:
# Get basic placement
zmin, zmax = Z.min(), Z.max()
Ltot = zmax - zmin
zmin, zmax, Ltot

In [ ]:
# Create a lume-impact style fieldmap and process into flat 1D rfdata
fmap = {
    "info": {"format": "solrf"},
    "field": {
        "Ez": {"z0": 0.0, "z1": 0.0, "L": 0.0, "fourier_coefficients": np.array([0.0])},
        "Bz": {"z0": 0, "z1": Ltot, "L": Ltot, "fourier_coefficients": fcoefs1},
    },
}


rfdata = fieldmaps.data_from_solrf_fieldmap(fmap)
rfdata

In [ ]:
# This simply writes 'data' this to a file
fieldmaps.write_fieldmap("templates/solenoid/rfdata666", fmap)

In [ ]:
zcenter = 0.24653  # Intended center

ele = {
    "description": "name:SOL1B",
    "L": Ltot,
    "type": "solrf",
    "zedge": zcenter + zmin,
    "rf_field_scale": 0.0,
    "rf_frequency": 0.0,
    "theta0_deg": 0.0,
    "filename": "rfdata666",
    "radius": 0.15,
    "x_offset": 0.0,
    "y_offset": 0.0,
    "x_rotation": 0.0,
    "y_rotation": 0.0,
    "z_rotation": 0.0,
    "solenoid_field_scale": 0.057,
    "name": "SOL1",
}
ele

In [ ]:
import impact

# This is the line to add to ImpactT.in
impact.lattice.ele_line(ele)

# New-style solRF field

In [ ]:
from pmd_beamphysics.fields.expansion import (
    fft_derivative_array,
    spline_derivative_array,
)
from impact.fieldmaps import run_RFcoef


from numpy import sqrt, exp


def gaussian_derivatives(z):
    f0 = 1 / sqrt(2 * pi) * exp(-(z**2) / 2)
    return np.array([1 * f0, (-z) * f0, (-1 + z**2) * f0, (3 * z - z**3) * f0]).T


ZZ = np.linspace(-6, 6, 1000)
DZ = np.diff(ZZ)[0]

AFZ = gaussian_derivatives(ZZ)
AFZ /= AFZ[:, 0].max()  # Normalize for comparisons

FZ = AFZ[:, 0]

N_COEF = 30
rfdatax2 = run_RFcoef(
    ZZ, FZ, n_coef=N_COEF, exe="$GITHUB/IMPACT-T/utilities/RFcoeflcls"
)["rfdatax2"][1:]
myrfdatax2 = fft_derivative_array(FZ, ncoef=N_COEF, dz=DZ)

myrfdatax2_spline = spline_derivative_array(ZZ, FZ)


ORDER = 0
plt.plot(abs(rfdatax2[:, ORDER] - AFZ[:, ORDER]), label="Fortran error")  #
plt.plot(abs(myrfdatax2[:, ORDER] - AFZ[:, ORDER]), "--", label="Python error, FFT")  #
plt.plot(
    abs(myrfdatax2_spline[:, ORDER] - AFZ[:, ORDER]), "--", label="Python error, Spline"
)  #
plt.legend()
plt.yscale("log")


rfdatax2.shape, myrfdatax2.shape

In [ ]:
def compare(order):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()

    f0 = AFZ[:, order]
    f1 = rfdatax2[:, order]
    f2 = myrfdatax2[:, order]
    f3 = myrfdatax2_spline[:, order]

    err1 = abs((f1 - f0) / f0)
    err2 = abs((f2 - f0) / f0)
    err3 = abs((f3 - f0) / f0)
    ax2.plot(f0, label="Reference", color="black")  #
    # ax2.plot( f1   , label='Fortran')#
    # ax2.plot( f2, '--', label='Python, FFT')#
    # ax2.plot( f3, '--', label='Python, Spline')#
    # ax2.legend(loc='upper right')

    ax.set_title(f"Gaussian derivative, order {order}")
    ax.set_ylabel("relative error")
    ax2.set_ylabel("True function")
    ax.plot(err1, label="Fortran RFcoefs")  #
    ax.plot(err2, "--", label="Python, FFT")  #
    ax.plot(err3, "--", label="Python, Spline")  #
    ax.set_yscale("log")
    ax.legend(loc="upper left")


compare(0)

In [ ]:
compare(1)

In [ ]:
compare(2)

In [ ]:
compare(3)